In [2]:
initial = torch.Tensor({
{1.0},
{0.0},
{0.0},
}):log()

In [4]:
transition = torch.Tensor({
{0.0, 0.0, 0.0},
{0.8, 0.4, 0.6},
{0.2, 0.6, 0.4}
}):log()

In [6]:
emission = torch.Tensor({
-- START, deal, fail, talks
{1.0, 0.0, 0.0},
{0.0, 0.45, 0.4},
{0.0, 0.1, 0.4},
{0.0, 0.45, 0.1}
}):log()

In [22]:
print(transition:size())
print(emission:size())

 3
 3
[torch.LongStorage of size 2]

 4
 3
[torch.LongStorage of size 2]



In [31]:
n

In [29]:
-- log-scores of transition and emission
-- corresponds to the vector y in the lecture notes
-- i: timestep for the computed score
function score_hmm(observations, i)
    print(emission[observations[i]])
    print(emission[observations[i]]:view(C, 1))
    local observation_emission = emission[observations[i]]:view(C, 1):expand(C, C)
    -- NOTE: allocates a new Tensor
    return observation_emission + transition
end

In [16]:
-- Viterbi algorithm.
-- observations: a sequence of observations, represented as integers
-- logscore: the edge scoring function over classes and observations in a history-based model
function viterbi(observations, logscore)
    local n = observations:size(1)
    local max_table = torch.Tensor(n, C)
    local backpointer_table = torch.Tensor(n, C)
    
    -- first timestep
    -- the initial most likely paths are the initial state distribution
    -- NOTE: another unnecessary Tensor allocation here
    local maxes, backpointers = (initial + emission[observations[1]]):max(2)
    max_table[1] = maxes
    
    -- remaining timesteps ("forwarding" the maxes)
    for i=2,n do
        -- precompute edge scores
        y = logscore(observations, i)
        scores = y + maxes:view(1, C):expand(C, C)
        -- compute new maxes (NOTE: another unnecessary Tensor allocation here)
        maxes, backpointers = scores:max(2)
        -- record
        max_table[i] = maxes
        backpointer_table[i] = backpointers
    end
    -- follow backpointers to recover max path
    local classes = torch.Tensor(n)
    maxes, classes[n] = maxes:max(1)
    for i=n,2,-1 do
        classes[i-1] = backpointer_table[{i, classes[i]}]
    end
    return classes
end

In [30]:
c = viterbi(torch.Tensor({1, 2, 4, 3}), score_hmm)

        -inf
 -7.9851e-01
 -9.1629e-01
[torch.DoubleTensor of size 3]

        -inf
 -7.9851e-01
 -9.1629e-01
[torch.DoubleTensor of size 3x1]

        -inf         -inf         -inf
 -1.0217e+00  -1.7148e+00  -1.3093e+00
 -2.5257e+00  -1.4271e+00  -1.8326e+00
[torch.DoubleTensor of size 3x3]

        -inf
 -7.9851e-01
 -2.3026e+00
[torch.DoubleTensor of size 3]

        -inf
 -7.9851e-01
 -2.3026e+00
[torch.DoubleTensor of size 3x1]

        -inf         -inf         -inf
 -1.0217e+00  -1.7148e+00  -1.3093e+00
 -3.9120e+00  -2.8134e+00  -3.2189e+00
[torch.DoubleTensor of size 3x3]

        -inf
 -2.3026e+00
 -9.1629e-01
[torch.DoubleTensor of size 3]

        -inf
 -2.3026e+00
 -9.1629e-01
[torch.DoubleTensor of size 3x1]

        -inf         -inf         -inf
 -2.5257e+00  -3.2189e+00  -2.8134e+00
 -2.5257e+00  -1.4271e+00  -1.8326e+00
[torch.DoubleTensor of size 3x3]

